In [0]:
import os
import shutil
import zipfile
import glob
from PIL import Image
from google.colab import drive, files

drive.mount('/content/drive')

path = '/content/drive/My Drive/video_data/kiminihare'
ret_name = 'kiminihare_fps1_256'
zip_name = 'kiminihare_fps1_select'
temp_dir = '/content/temp'
os.makedirs(temp_dir)

In [0]:
with zipfile.ZipFile('{}/{}.zip'.format(path, zip_name)) as existing_zip:
    existing_zip.extractall(path)

In [0]:
files = glob.glob('{}/{}/*.jpg'.format(path, zip_name))

In [0]:
with zipfile.ZipFile('{}/{}.zip'.format(path, ret_simple), 'w', compression=zipfile.ZIP_DEFLATED) as new_zip:
    for file in files:
        new_path = '{}/{}'.format(temp_dir, file.split('/')[-1])
        try:
            img = Image.open(file)
            #img = pad_and_square(img, background_color=(255, 255, 255), size=(256, 256))
            img = simple_resize(img, size=(256, 256))
            img.save(new_path)
            new_zip.write(new_path, arcname=file.split('/')[-1])
        except Exception as e:
            print(e)

In [0]:
shutil.rmtree(temp_dir)
shutil.rmtree('{}/{}'.format(path, zip_name))
if os.path.exists('{}/__MACOSX'.format(path)):
    shutil.rmtree('{}/__MACOSX'.format(path))

In [0]:
def crop_mid(img, shift=0, background_color=(255, 255, 255), size=(256, 256)):
    w, h = img.size
    if w == h:
        im = img.resize(size, resample=Image.LANCZOS)
    elif w > h:
        im =  Image.new(img.mode, (h, h), background_color)
        im.paste(img, (-((w - h) // 2) + shift, 0))
        im = im.resize(size, resample=Image.LANCZOS)
    elif w < h:
        im =  Image.new(img.mode, (w, w), background_color)
        im.paste(img, (0, -(h - w) // 2))
        im = im.resize(size, resample=Image.LANCZOS)
    return im

# assume width > height    

def crop_along_right_edge(img, shift=0, background_color=(255, 255, 255), size=(256, 256)):
    w, h = img.size
    im =  Image.new(img.mode, (h, h), background_color)
    im.paste(img, (-(w - h) + shift, 0)) 
    im = im.resize(size, resample=Image.LANCZOS)
    return im

def crop_along_left_edge(img, shift=0, background_color=(255, 255, 255), size=(256, 256)):
    w, h = img.size
    im =  Image.new(img.mode, (h, h), background_color)
    im.paste(img, (0 + shift, 0))    
    im = im.resize(size, resample=Image.LANCZOS)
    return im

def pad_and_square(img, background_color, size):
    width, height = img.size
    if width == height:
        im = img.resize(size, resample=Image.LANCZOS)
    elif width > height:
        im = Image.new(img.mode, (width, width), background_color)
        im.paste(img, (0, (width - height) // 2))
        im = im.resize(size, resample=Image.LANCZOS)
    else:
        im = Image.new(img.mode, (height, height), background_color)
        im.paste(img, ((height - width) // 2, 0))
        im = im.resize(size, resample=Image.LANCZOS)
    return im

def simple_resize(img, size=(256, 256)):
    return img.resize(size, resample=Image.LANCZOS)

def concat_images(im1, im2):
    ret = Image.new('RGB', (im1.width + im2.width, im1.height))
    ret.paste(im1, (0, 0))
    ret.paste(im2, (im1.width, 0))
    return ret